In [ ]:
import numpy as np
import numpy as np
import pandas as  pd
import tensorflow as tf
from tensorflow import keras
#import keras
import matplotlib.pyplot as plt 
from tensorflow.keras import layers
from sklearn import *
import math
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import scipy
tf.autograph.set_verbosity(0)
from sklearn import linear_model as lm  # Used for solving linear regression problems
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import cmath
import math

In [ ]:
def wrap(phases): 
    ''' It wrappes an already unwrapped time series angular phase
    
     Inputs
       phases: Vector of unwrapped angular phase
  
     Outputs
       wrapped Time series angular phase
      '''
  
    return (phases + np.pi) % (2 * np.pi) - np.pi

In [2]:
def reIm2complex_sb(sbi):
    
    ''' It takes a matrix of 1 subband but Re and Im part concatenated. It merges 
       them together into a complex subband of MPCs
  
       Inputs
       sbi: matrix [tti x p·2]
  
       Outputs
       sbfold: matrix [tti x p]
       '''
  

    ncol_out = int(sbi.shape[1]/2)
    nrow_out = int(sbi.shape[0])

    sbfold = np.empty([(nrow_out), (ncol_out)], dtype = 'complex_')
    sbi = sbi.values
    for i in range((ncol_out)):

        sbfold[:, i] = np.array([r * (math.cos(phi) + math.sin(phi)*1j) for r, phi in zip(sbi[:, i], sbi[:, i+ncol_out])])
        
    return sbfold

In [ ]:
def predict_ar_1step(theta, y_target):
    """Predicts the value y_t for t = p+1, ..., n, for an AR(p) model, based on the data in y_target using
    one-step-ahead prediction.

    :param theta: array (p,), AR coefficients, theta=(a1,a2,...,ap).
    :param y_target: array (n,), the data points used to compute the predictions.
    :return y_pred: array (n-p,), the one-step predictions (\hat y_{p+1}, ...., \hat y_n) 
    """

    n = len(y_target)
    p = theta.shape[1]
    
    # Number of steps in prediction
    m = n-p
    y_pred = np.zeros(m+1)
    for i in range(m):
        y_pred[i] =  (np.flip(y_target[i:i+p]) * theta).sum()
        
    return y_pred

In [ ]:
def fit_ar(y, p):
    """Fits an AR(p) model. The loss function is the sum of squared errors from t=p+1 to t=n.

    :param y: array (n,), training data points
    :param p: int, AR model order
    :return theta: array (p,), learnt AR coefficients
    """

    # Number of training data points
    n = y.shape[0]
    
    # Construct the regression matrix
    Phi = np.zeros((n-p, p))# <COMPLETE THIS LINE>
    for j in range(p):
        Phi[:,j] = y[(p-(j+1)): (n-(j+1)), 0] # <COMPLETE THIS LINE>
    
    # Drop the first p values from the target vector y
    yy = y[p:]  # yy = (y_{t+p+1}, ..., y_n)

    # Here we use fit_intercept=False since we do not want to include an intercept term in the AR model
    regr = lm.LinearRegression(fit_intercept=False)
    regr.fit(Phi,yy)    

    return regr.coef_

In [ ]:
def NMSE(true_chan, pred_chan):
    """Computes Normalised Mean Squared Error. Uses the hermitian to compute Norm 2

    :param true_chan: [Ncarriers x Nantennas] Matrix
    :param pred_chan: [Ncarriers x Nantennas] Matrix
    :return: NMSE
    """
    chann_diff = true_chan - pred_chan
    
    num = (np.sqrt(sum(sum(((chann_diff.conj().T).dot(chann_diff))**2)))).real
    den = (np.sqrt(sum(sum(((true_chan.conj().T).dot(true_chan))**2)))).real
    
    return num/den
 